#  메타데이터 분석하기 (1)

**[목차]**
* 1개의 주행 데이터 파싱하기 

## 1. 1개의 주행 데이터 파싱하기
주행 데이터는 JSON,  Image 등 다양한 파일로 구성되어 있으며 이러한 파일들은 주행 시나리오 하나에 1개의 폴더에 저장되어 있음

특히, JSON 파일은 자료를 유연하게 저장할 수 있는 장점으로 인해 **"메타데이터(MetaData)"** 와 **"어노테이션 데이터(Annotation Data)"** 로 구성되어 있음
* **메타데이터(MetaData) :** 차량의 센서 데이터, 위치, 주변 환경의 특성 등 차량의 운행 상황과 관련된 데이터
* **어노테이션 데이터(Annotation Data) :** 촬영된 이미지 내의 다양한 개체들(차량, 표지판 등)에 대한 위치, 유형, 크기 등과 관련된 데이터


### 1.1 JSON 데이터 불러오기
해당 실습에서는 1개의 주행 데이터(JSON)를 파싱하고 데이터가 정확한지 혹은 어떤 의미를 찾을 수 있는지 확인하기 위해 다양한 라이브러리를 활용하여 간단한 검증을 진행하는 실습을 진행

분석하고자 하는 주행 데이터는 실제 수집된 주행 데이터 중에서 임의로 하나를 선정하였음

In [ ]:
path = "./data/driving_data.json"

In [ ]:
import json

with open(path, "r") as f :
    data = json.load(f)

data

### 1.2 데이터 분리하기
메타데이터와 어노테이션 데이터가 같이있어서 보기가 복잡하고 불필요할 수도 있기 때문에 해당 데이터를 분리하는 작업을 진행

불러온 JSON 데이터(data)는 `dictionary(dict)` 타입이므로 이 특성을 이용

In [ ]:
type(data)

In [ ]:
annotation_data = data["annotations"]
annotation_data

In [ ]:
del data["annotations"]

In [ ]:
meta_data = data

meta_data

### 1.3 필요한 센서 데이터 파싱하기

메타데이터 안에는 많은 정보들이 있지만 그 중에서 직관적이고 분석에 도움이 되는 데이터를 위주로 선정하여 분석을 진행

| **메타데이터 항목** | **설명** |
| --- | --- |
| unix_timestamp | 파일이 생성된 시각 (unix time stamp) |
| gps_latitude | GPS 위도 |
| gps_longitude | GPS 경도 |
| yaw | 요 |
| roll | 롤 |
| pitch | 피치 |
| long_velocity | 경도 방향의 속도 |
| lat_velocity | 위도 방향의 속도 |
| long_accel | 전진 또는 후진 방향으로 가속하거나 감속하는 정도(가속도) |
| lat_accel | 좌우로 가속하거나 감속하는 정도(가속도) |



#### (1) Unix Time Stamp

* **Unix Time Stamp** 는 1970년 1월 1일 00:00:00 (UTC)로 부터 경과한 시간을 초 단위로 나타내는 숫자 값

* POSIX 시간 또는 Epoch 시간이라고도 부름

* 시간을 전 세계적으로 일관되게 표현할 수 있어서 데이터 동기화, 이벤트 추적 등에서 유용하게 사용할 수 있음


메타데이터에 기록된 Unix Time Stamp는 `"unix_timestamp"`에 저장되어 있으며 확인한 결과는 다음과 같음

In [ ]:
unix_timestamp = meta_data["unix_timestamp"]
unix_timestamp

참고로 현재 시간의 Unix Time Stamp을 확인하는 방법은 내장 라이브러리인 `time` 라이브러리를 사용하면 확인 할 수 있음

In [ ]:
import time

epoch_time = time.time()
epoch_time

Unix Time Stamp를 우리가 아는 시간(예. 2022년 2월 2일 2시 22분 22초)으로 변환하기 위해서는 `datetime` 이라는 내장 라이브러리 필요

* **fromtimestamp :** Unix Time Stamp에 해당하는 날짜를 반환해주는 메소드
* **strftime :** 지정한 포맷(예. %Y : YYYY)으로 날짜 자료를 생성해주는 메소드

In [ ]:
from datetime import datetime
 
dt_object = datetime.fromtimestamp(epoch_time)

real_time = dt_object.strftime("%Y-%m-%d %H:%M:%S")
real_time

소수점을 제외한 상태에서 현재 시간과 메타데이터의 시간을 비교하면 자릿수가 다른 것을 확인 할 수 있음

In [ ]:
print(f"현재 시간 : {int(epoch_time)}")
print(f"측정 시간 : {int(unix_timestamp)}")

즉, 메타데이터의 unix_timestamp는 초(second) 단위가 아니라 밀리초(milli second)로 저장되어 있다는 것을 확인할 수 있음


<b><p style="text-align: center"> 1 second = 1000 millisecond </p></b>

밀리초 단위를 사용하여 `fromtimestamp`를 사용하면 에러가 발생하기 때문에 초 단위로 맞춰주는 작업을 통해 변환

In [ ]:
dt_object = datetime.fromtimestamp(unix_timestamp/1000)

readable_time = dt_object.strftime("%Y-%m-%d %H:%M:%S")
readable_time

해당 데이터는 2022년 6월 24일 12시 반에 생성된 자료임을 확인 할 수 있음

메타데이터에는 이를 직접적으로 확인할 데이터를 가지고 있지 않지만 `parent_path` 내의 데이터를 확인하면 동일한 시간이 있는 것을 확인 할 수 있음

(현재 실행환경(런박스)의 타임존(tz)이 한국으로 설정되어 있지 않아 약 9시간 정도 차이가 있습니다.)

In [ ]:
meta_data["parent_path"]

#### (2) 위도와 경도

GPS 정보를 이용하면 위도와 경도 데이터를 얻을 수 있으며 이 데이터를 이용하면 지구 상에 어느 위치에 있는지 확인 할 수 있음

* **위도 :** 지구의 수평선(적도)을 기준으로 위치를 표시하는 방식
* **경도 :** 지구의 수직선(그리니치 천문대)을 기준으로 위치를 표시하는 방식

메타데이터에 기록된 위도와 경도는 `"gps_latitude"`, `"gps_longitude"`에 저장되어 있으며 확인한 결과는 다음과 같음

In [ ]:
latitude = meta_data["gps_latitude"]
longitude = meta_data["gps_longitude"]

gps_point = f"{latitude}, {longitude}"
gps_point

위도, 경도를 이용한 위치 정보를 바탕으로 위치를 확인하기 위해서는 `geopy` 라는 외부 라이브러리가 필요

해당 라이브러리의 `reverse` 메소드에 현재 위치 정보를 넣으면 위치의 국가, 주소, 우편번호 등의 정보를 얻을 수 있음

In [ ]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="search my location") 

elice_location = geolocator.reverse("37.503762, 127.048918")
elice_location

In [ ]:
vehicle_location = geolocator.reverse(gps_point)
vehicle_location

location 객체에 `raw` 를 이용하면 해당 위치의 raw data를 확인 할 수 있음

In [ ]:
raw_data = vehicle_location.raw  
raw_data

해당 위치의 주소 정보를 얻고 싶다면 `address` 를 이용

In [ ]:
address = vehicle_location.address
address

메타데이터에서 나타내는 지리 정보는 `region_name`과 `location` 이 있지만 현재 주소 체계가 아닌 구 주소 체계임을 고려할 필요가 있음

In [ ]:
meta_data["region_name"], meta_data["location"]

위치 정보를 지도 위로 시각화하기 위해서는 `folium` 이라는 외부 라이브러리 필요

In [ ]:
import folium   

# 지도 객체 생성 (초기 위치와 확대 수준 설정)  
mymap = folium.Map(location=[latitude, longitude], zoom_start=50)  

# 마커 추가  
folium.Marker([latitude, longitude], popup='Marker at specified location').add_to(mymap)  

# 또는 주피터 노트북 환경에서 직접 지도를 표시할 수 있음  
mymap

#### (3) 조향 및 속도 관련 데이터

메타데이터 안에는 차량의 다양한 조향과 속도 데이터를 가지고 있음 

![](img/yaw_roll_pitch.png)


**[조향 관련]**
* **yaw (요) :** 차량의 수직축을 따라 발생하는 회전 운동
* **roll (롤) :** 차량의 길이축을 따라 발생하는 회전 운동
* **pitch (피치) :** 차량의 횡축(가로축)을 따라 발생하는 회전 운동


**[속도 관련]**
* **long_velocity :** 전진 방향으로의 속도
* **long_accel :** 전진 방향으로의 가속도
* **lat_velocity :** 옆면 방향으로의 속도
* **lat_accel :** 옆면 방향으로의 가속도


이 데이터를 이용하면 차량의 방향, 현재 상태 등 다양한 정보를 확인 할 수 있음

하지만 현재의 데이터는 단일 데이터이므로 뚜렷한 의미를 찾을 수 없음

의미있는 데이터를 도출하기 위해서는 단일 데이터가 아닌 시간에 따른 여러 조향 및 속도 데이터가 필요

현재 실습에서는 단일 메타 데이터에서 관련 정보를 파싱하고 이를 데이터프레임으로 정리하는 방법을 진행

In [ ]:
yaw = meta_data["yaw"]
roll = meta_data["roll"]
pitch = meta_data["pitch"]

print(f"Yaw : {yaw}, Roll : {roll}, Pitch : {pitch}")

In [ ]:
long_velocity = meta_data["long_velocity"]
long_accel = meta_data["long_accel"]

lat_velocity = meta_data["lat_velocity"]
lat_accel = meta_data["lat_accel"]

print(f"전진 방향 속도: {long_velocity}, 가속도 : {long_accel}")
print(f"옆면 방향 속도: {lat_velocity}, 가속도 : {lat_accel}")

In [ ]:
import pandas as pd

df = pd.DataFrame([meta_data])
df

In [ ]:
df[["yaw", "roll", "pitch"]]

In [ ]:
df[["long_velocity", "long_accel", "lat_velocity", "lat_accel"]]

### [TODO] 다른 시나리오의 단일 주행 데이터 파싱하기
* data 폴더에 있는 `parsing_target.json` 은 지금까지 분석한 주행 데이터와 <u>다른 시나리오에서 추출한 데이터 중 하나</u>입니다. 
* 해당 데이터를 파싱하여 다양한 메타데이터를 분석하는 코드를 작성하세요.

먼저 JSON 파일을 불러오는 코드를 작성해 주세요.

In [ ]:
json_path = "data/parsing_target.json"


# JSON 파일을 Dictionary 객체로 불러오는 코드를 작성해 주세요.
with open(None) as f :
    parsed_data = None


parsed_data

원할한 분석을 위해 메타 데이터만 저장하는 코드를 작성해 주세요.

In [ ]:
del None


mission_meta = None
mission_meta

메타데이터를 이용하여 데이터프레임(mission_df)을 만드는 코드를 작성해 주세요.

데이터프레임의 컬럼과 내용은 다음과 같습니다. 컬럼의 순서가 다르면 오답으로 처리될 수 있습니다. 순서를 꼭 확인해 주세요.

|create_time| gps_latitude | gps_longitude | address | yaw | roll | pitch | long_velocity | lat_velocity | long_accel | lat_accel | 
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | 
| 파일이 생성된 시간 </br> 형식 : YYYY-MM-DD HH:MM:SS | GPS 위도 | GPS 경도 | 지리 주소 정보 </br> 예 : '성남대로, 분당구, 성남시, 경기도, 13439, 대한민국' | 요 정보 | 롤 정보 | 피치 정보 | 전진 방향으로의 속도 정보 | 옆면 방향으로의 속도 정보 | 전진 방향으로의 가속도 정보 | 옆면 방향으로의 가속도 정보 | 


#### (1) 데이터프레임 생성하기

메타데이터를 데이터프레임으로 생성하고 원하는 데이터만 추출하는 코드를 작성하세요.

In [ ]:
# 메타데이터를 데이터프레임으로 변환하는 코드를 작성하세요.
meta_df = None


# 위에서 지시한 컬럼의 이름을 확인하세요.
# 필요한 컬럼이 있다면 리스트 안에 추가하고 불필요한 컬림이 있다면 삭제하세요.
select_col_name = [
                    "unix_timestamp", 
                    "gps_latitude", 
                    "gps_longitude", 
                    "location", 
                    "season",
                    "yaw", 
                    "roll", 
                    "pitch", 
                    "long_velocity", 
                    "lat_velocity", 
                    "long_accel"
                ]


# 데이터프레임에서 데이터를 추출하는 코드를 작성하세요.
mission_df = None


# 추출한 데이터프레임을 확인하는 코드입니다.
mission_df

#### (2) 데이터 변환하기

이제 추출된 지시사항에 맞게 수정하는 작업을 진행하겠습니다. 먼저 시간 데이터를 변환하겠습니다.

위에서 학습한 내용을 이용하여 unix time stamp를 변환하는 코드를 작성하세요.

In [ ]:
# datetime 라이브러리를 이용하여 unix time stamp의 시간을 변환하는 코드를 작성하세요.
dt_object = None


readable_time = None
readable_time

Pandas에서 데이터프레임 내의 값을 변경하는 방법은 다양합니다. 

그 중에서 `at`을 이용하면 **지정한 위치의 하나의 값**을 추출하거나 변경 할 수 있습니다. 

`at`의 사용방법은 다음과 같습니다.

In [ ]:
# Padnas at 사용법 예시
at_df = pd.DataFrame([[1,2], 
                   [3,4]], 
                   index=['row1', 'row2'], 
                   columns=['col1', 'col2']
                )


# at을 이용하여 추출하기
result = at_df.at['row1', 'col2']
print("result : ", result)


# at을 이용하여 변경하기
at_df.at['row2', 'col1'] = 'Rabbit'
print(at_df)

본인이 아는 방법을 이용하여 데이터프레임의 값을 변경하세요.

In [ ]:
# unix_timestamp 의 값을 변경한 데이터로 변환하는 코드를 작성하세요.
None


# 변경된 데이터프레임을 확인하는 코드입니다.
mission_df

데이터프레임에서 위도와 경도를 추출하여 주소 데이터를 생성하는 코드를 작성하겠습니다. 

본인이 아는 방법을 이용해서 위도와 경도 데이터를 변수에 저장하세요.

In [ ]:
# 각 변수에 위도와 경도 데이터를 저장하는 코드를 작성하세요.
mission_latitude = None
mission_longitude = None


# 위도와 경도를 확인하는 코드입니다.
print(mission_latitude, mission_longitude)

참고로 이런 방법도 있습니다.

In [ ]:
mission_latitude, mission_longitude = mission_df[["gps_latitude", "gps_longitude"]].values[0]

print(mission_latitude, mission_longitude)

추출된 위도와 경도 데이터를 이용하여 주소 데이터를 생성하세요.

In [ ]:
# 주소를 검색 할 수 있도록 위도와 경도 데이터를 빈 칸 안에 입력하세요.
mission_gps_point = f"{None}, {None}"


# 해당 데이터를 이용해서 주소 데이터를 얻는 코드를 작성하세요.
vehicle_location = None
mission_address = None


# 주소 데이터를 확인하는 코드입니다.
mission_address

In [ ]:
# location 의 값을 변경한 데이터로 변환하는 코드를 작성하세요.
None


# 변경된 데이터프레임을 확인하는 코드입니다.
mission_df

#### (3) 제출 파일 준비하기 

현재의 데이터프레임은 지시사항과 컬럼 이름의 차이가 있습니다. 

마지막으로 컬럼 이름을 지시사항에 맞게 변경하는 코드를 이용하여 컬럼의 이름을 변경하고 제출을 위해 CSV로 저장하는 코드를 작성하세요.

In [ ]:
# 컬럼의 이름을 변경하는 코드를 작성하세요.
# unix_timestamp → create_time 으로 location → address 로 변경해 주세요.
mission_df = None


# 변경된 데이터프레임을 확인하는 코드입니다.
mission_df

마지막으로 최종 제출을 위해 지금까지 다룬 데이터프레임을 CSV로 저장하는 코드를 작성해 주세요. 

저장되는 CSV의 이름은 `result.csv` 여야 하며 인덱스는 저장되면 안됩니다. 

In [ ]:
None